In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [36]:
import pandas as pd
import pickle as pkl

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
%run utils.ipynb
%run targets.ipynb

In [4]:
file = 'data/ETHBTC_1m_1519496760000_1549568220000.csv'

In [5]:
# Get Data
df = pd.read_csv(file)
df.shape
df.head(10)

(500000, 12)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  VOLUME  \
0  1519496760000  0.085541  0.085570  0.085407  0.085473  27.714   
1  1519496820000  0.085473  0.085570  0.085448  0.085449  20.128   
2  1519496880000  0.085448  0.085569  0.085400  0.085538  45.510   
3  1519496940000  0.085538  0.085546  0.085402  0.085540  22.231   
4  1519497000000  0.085539  0.085552  0.085464  0.085464  15.243   
5  1519497060000  0.085466  0.085555  0.085464  0.085545  31.895   
6  1519497120000  0.085545  0.085545  0.085464  0.085539  13.325   
7  1519497180000  0.085539  0.085555  0.085414  0.085517  28.202   
8  1519497240000  0.085460  0.085545  0.085419  0.085534  15.302   
9  1519497300000  0.085440  0.085545  0.085440  0.085533  13.730   

      CLOSE_TIME  QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519496819999   2.370345         192         15.308        1.309733      0  
1  1519496879999   1.720609         122          6.579        0.562654      0  
2  1519496939999   3.890176         209         19.426        1.660960      0  
3  1519496999999   1.900513         105         14.320        1.224483      0  
4  1519497059999   1.303412         117          8.373        0.716219      0  
5  1519497119999   2.727413         131         19.237        1.645547      0  
6  1519497179999   1.139346          78          7.311        0.625352      0  
7  1519497239999   2.411093         195         14.409        1.232346      0  
8  1519497299999   1.307959         134          7.216        0.617035      0  
9  1519497359999   1.173580         105          6.186        0.529001      0

In [7]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 5)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE
0  1519496760000  0.085541  0.085570  0.085407  0.085473
1  1519496820000  0.085473  0.085570  0.085448  0.085449
2  1519496880000  0.085448  0.085569  0.085400  0.085538
3  1519496940000  0.085538  0.085546  0.085402  0.085540
4  1519497000000  0.085539  0.085552  0.085464  0.085464

In [8]:
data_validation(data)

In [9]:
# Get Target
%run targets.ipynb
target = get_target_1(file)
target.shape
target.head()

(499999, 2)

OPEN_TIME  TARGET
0  1519496760000     1.0
1  1519496820000     0.0
2  1519496880000     0.0
3  1519496940000     1.0
4  1519497000000     0.0

In [10]:
data = pd.merge(data, target, on='OPEN_TIME')
data.shape
data.head()

(499999, 6)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  TARGET
0  1519496760000  0.085541  0.085570  0.085407  0.085473     1.0
1  1519496820000  0.085473  0.085570  0.085448  0.085449     0.0
2  1519496880000  0.085448  0.085569  0.085400  0.085538     0.0
3  1519496940000  0.085538  0.085546  0.085402  0.085540     1.0
4  1519497000000  0.085539  0.085552  0.085464  0.085464     0.0

In [12]:
data_validation(data)

In [13]:
data.describe()

OPEN_TIME           OPEN           HIGH            LOW  \
count  4.999990e+05  499999.000000  499999.000000  499999.000000   
mean   1.534528e+12       0.051343       0.051377       0.051307   
std    8.686686e+09       0.019869       0.019885       0.019853   
min    1.519497e+12       0.024659       0.024666       0.024630   
25%    1.526997e+12       0.032582       0.032597       0.032566   
50%    1.534530e+12       0.045459       0.045491       0.045427   
75%    1.542043e+12       0.071582       0.071626       0.071534   
max    1.549568e+12       0.089300       0.089800       0.089269   

               CLOSE         TARGET  
count  499999.000000  499999.000000  
mean        0.051342       0.479845  
std         0.019869       0.499594  
min         0.024661       0.000000  
25%         0.032583       0.000000  
50%         0.045460       0.000000  
75%         0.071582       1.000000  
max         0.089330       1.000000

In [14]:
frac_data_use = 0.2

tr_frac = 0.6
cv_frac = 0.2
tt_frac = 0.2

TR_data = data[:int(len(data)*tr_frac)]
CV_data = data[int(len(data)*tr_frac):int(len(data)*(tr_frac+cv_frac))]
TT_data = data[int(len(data)*(tr_frac+cv_frac)):]

TR_data.shape
CV_data.shape
TT_data.shape

(299999, 6)

(100000, 6)

(100000, 6)

In [120]:
win=300
TR_X, TR_y = get_multiple_ts_dataset(TR_data.values, tr_win=win, tt_win=0, point_target=True)
CV_X, CV_y = get_multiple_ts_dataset(CV_data.values, tr_win=win, tt_win=0, point_target=True)
TT_X, TT_y = get_multiple_ts_dataset(TT_data.values, tr_win=win, tt_win=0, point_target=True)

In [121]:
TR_X.shape
TR_y.shape

(299700, 1201)

(299700,)

In [122]:
TR_X_ = TR_X[:, 1:].T
CV_X_ = CV_X[:, 1:].T
TT_X_ = TT_X[:, 1:].T

TR_X_ = (TR_X_ - TR_X_.min(axis=0)) / (TR_X_.max(axis=0) - TR_X_.min(axis=0))
CV_X_ = (CV_X_ - CV_X_.min(axis=0)) / (CV_X_.max(axis=0) - CV_X_.min(axis=0))
TT_X_ = (TT_X_ - TT_X_.min(axis=0)) / (TT_X_.max(axis=0) - TT_X_.min(axis=0))

TR_X_[np.isnan(TR_X_)] = 0
CV_X_[np.isnan(CV_X_)] = 0
TT_X_[np.isnan(TT_X_)] = 0

TR_X[:, 1:] = TR_X_.T
CV_X[:, 1:] = CV_X_.T
TT_X[:, 1:] = TT_X_.T

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


In [123]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge(
    n_iter=300, 
    tol=0.00001, 
    alpha_1=1e-06, 
    alpha_2=1e-06, 
    lambda_1=1e-06, 
    lambda_2=1e-06, 
    compute_score=False, 
    fit_intercept=True, 
    normalize=False, 
    copy_X=True, 
    verbose=True
)

model = model.fit(TR_X[:, 1:], TR_y)

TR_y_pred = model.predict(TR_X[:, 1:])
CV_y_pred = model.predict(CV_X[:, 1:])
TT_y_pred = model.predict(TT_X[:, 1:])

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(CV_metrics, name='CV'))
res = res.append(pd.Series(TT_metrics, name='TT'))

print(res)

Convergence after  89  iterations
         acc       auc        f1   log_loss
TR  0.521815  0.502954  0.069680  16.515933
CV  0.522211  0.499654  0.048688  16.502244
TT  0.514669  0.499496  0.049314  16.762755


In [124]:
TR_y_pred = model.predict(TR_X[:, 1:])
TR_y_pred[:5]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TR_metrics

array([0.47546675, 0.473921  , 0.47798764, 0.48153715, 0.47732952])

{'acc': 0.5218151484818151,
 'auc': 0.5029539515290485,
 'f1': 0.06968048504992015,
 'log_loss': 16.515932732516685}

In [125]:
(TR_y_pred > 0.5).sum()

10266

In [126]:
CV_y_pred = model.predict(CV_X[:, 1:])
CV_y_pred[:5]

CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics

array([0.47194425, 0.46830877, 0.47024308, 0.47083005, 0.46621108])

{'acc': 0.5222114121222455,
 'auc': 0.4996536785502691,
 'f1': 0.04868794184606782,
 'log_loss': 16.502244244226503}

In [127]:
TT_y_pred = model.predict(TT_X[:, 1:])
TT_y_pred[:5]

TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics

array([0.47465472, 0.47133739, 0.46768741, 0.46859747, 0.46881043])

{'acc': 0.5146688598910744,
 'auc': 0.499495587215216,
 'f1': 0.04931431490431845,
 'log_loss': 16.76275487344526}

In [128]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL', 
    'diff':False,
    'rollNorm': True,
    'win':win
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': 'BayesR',
    'n_iter': 89,
    'accuracy':'{:.4f}'.format((CV_metrics['acc']+TT_metrics['acc'])/2)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

f = open('models/'+time_tag+'.pkl', 'wb')
pkl.dump(model, f, -1)
f.close()

%notebook -e models/{big_tag}.ipynb